#  Unit 2.4a Using Programs with Data, SQLAlchemy
> Using Programs with Data is focused on SQL and database actions.  Part A focuses on SQLAlchemy and an OOP programming style,
- toc: true
- image: /images/python.png
- categories: []
- type: ap
- week: 26

# Database and SQLAlchemy
> In this blog we will explore using programs with data, focused on Databases.  We will use SQLite Database to learn more about using Programs with Data.  ***Use Debugging through these examples to examine Objects created in Code***.

- College Board talks about ideas like 
    - Program Usage. "iterative and interactive way when processing information"
    - Managing Data.  "classifying data are part of the process in using programs", "data files in a Table"
    - Insight "insight and knowledge can be obtained from ...  digitally represented information"
    - Filter systems. 'tools for finding information and recognizing patterns"
    - Application. "the preserve has two databases", "an employee wants to count the number of book"

- PBL, Databases, Iterative/OOP
    - Iterative. Refers to a sequence of instructions or code being repeated until a specific end result is achieved
    - OOP. A computer programming model that organizes software design around data, or objects, rather than functions and logic
    - SQL. Structured Query Language, abbreviated as SQL, is a language used in programming, managing, and structuring data 


## Imports and Flask Objects
> Defines and key object creations

- Comment on where you have observed these working?  Provide a defintion of purpose.
    1. Flask app object
     - Flask creates the app object
     - App is getting assigned the attributes of the object
     - sets up for templete and database objects
    2. SQLAlchemy db object
     - db = SQLAlchemy()
     - sets up database project with properties from SQLAlchemy


In [66]:
"""
These imports define the key objects
"""

from flask import Flask
from flask_sqlalchemy import SQLAlchemy

"""
These object and definitions are used throughout the Jupyter Notebook.
"""

# Setup of key Flask object (app)
app = Flask(__name__)
# Setup SQLAlchemy object and properties for the database (db)
database = 'sqlite:///Shruthi.db'  # path and filename of database
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SQLALCHEMY_DATABASE_URI'] = database
app.config['SECRET_KEY'] = 'SECRET_KEY'
db = SQLAlchemy()


# This belongs in place where it runs once per project
db.init_app(app)


## Model Definition
> Define columns, initialization, and CRUD methods for users table in sqlite.db

- Comment on these items in the class, purpose and defintion.
    - class User: defines templete for users
    - db.Model inheritance: inherit ablility to make our class using database properties, methods, and attributes
    - _init_ method
        - helps create an object from a user class
        - creates a u1 object using attributes
    - ```@property```, ```@<column>.setter```
        - used to see what is inside an object and is used to change what is inside the object
        - setter allows us to change attributes
        - property shows us attributes (ex. value name= name is shown)
        - used to set and gain data
    - create, read, update, delete methods
        - methods inside of a class
        - interacts with the database
        - creates a new user with attributes
        - reads a user(s) and their attributes
        - updates a user(s) and their attributes
        - deletes a user(s) and their attributes

When executing the cell above, there is no result because the templete is created but the data is not created

In [67]:
""" database dependencies to support sqlite examples """
import datetime
from datetime import datetime
import json

from sqlalchemy.exc import IntegrityError
from werkzeug.security import generate_password_hash, check_password_hash


''' Tutorial: https://www.sqlalchemy.org/library.html#tutorials, try to get into a Python shell and follow along '''

# Define the User class to manage actions in the 'users' table
# -- Object Relational Mapping (ORM) is the key concept of SQLAlchemy
# -- a.) db.Model is like an inner layer of the onion in ORM
# -- b.) User represents data we want to store, something that is built on db.Model
# -- c.) SQLAlchemy ORM is layer on top of SQLAlchemy Core, then SQLAlchemy engine, SQL
class User(db.Model):
    __tablename__ = 'users'  # table name is plural, class name is singular

    # Define the User schema with "vars" from object
    id = db.Column(db.Integer, primary_key=True)
    _name = db.Column(db.String(255), unique=False, nullable=False)
    _uid = db.Column(db.String(255), unique=True, nullable=False)
    _password = db.Column(db.String(255), unique=False, nullable=False)
    _dob = db.Column(db.Date)

    # constructor of a User object, initializes the instance variables within object (self)
    def __init__(self, name, uid, password="123qwerty", dob=datetime.today()):
        self._name = name    # variables with self prefix become part of the object, 
        self._uid = uid
        self.set_password(password)
        if isinstance(dob, str):  # not a date type     
            dob = date=datetime.today()
        self._dob = dob

    # a name getter method, extracts name from object
    @property
    def name(self):
        return self._name
    
    # a setter function, allows name to be updated after initial object creation
    @name.setter
    def name(self, name):
        self._name = name
    
    # a getter method, extracts uid from object
    @property
    def uid(self):
        return self._uid
    
    # a setter function, allows uid to be updated after initial object creation
    @uid.setter
    def uid(self, uid):
        self._uid = uid
        
    # check if uid parameter matches user id in object, return boolean
    def is_uid(self, uid):
        return self._uid == uid
    
    @property
    def password(self):
        return self._password[0:10] + "..." # because of security only show 1st characters

    # update password, this is conventional method used for setter
    def set_password(self, password):
        """Create a hashed password."""
        self._password = generate_password_hash(password, method='sha256')

    # check password parameter against stored/encrypted password
    def is_password(self, password):
        """Check against hashed password."""
        result = check_password_hash(self._password, password)
        return result
    
    # dob property is returned as string, a string represents date outside object
    @property
    def dob(self):
        dob_string = self._dob.strftime('%m-%d-%Y')
        return dob_string
    
    # dob setter, verifies date type before it is set or default to today
    @dob.setter
    def dob(self, dob):
        if isinstance(dob, str):  # not a date type     
            dob = date=datetime.today()
        self._dob = dob
    
    # age is calculated field, age is returned according to date of birth
    @property
    def age(self):
        today = datetime.today()
        return today.year - self._dob.year - ((today.month, today.day) < (self._dob.month, self._dob.day))
    
    # output content using str(object) is in human readable form
    # output content using json dumps, this is ready for API response
    def __str__(self):
        return json.dumps(self.read())

    # CRUD create/add a new record to the table
    # returns self or None on error
    def create(self):
        try:
            # creates a person object from User(db.Model) class, passes initializers
            db.session.add(self)  # add prepares to persist person object to Users table
            db.session.commit()  # SqlAlchemy "unit of work pattern" requires a manual commit
            return self
        except IntegrityError:
            db.session.remove()
            return None

    # CRUD read converts self to dictionary
    # returns dictionary
    def read(self):
        return {
            "id": self.id,
            "name": self.name,
            "uid": self.uid,
            "dob": self.dob,
            "age": self.age,
        }

    # CRUD update: updates user name, password, phone
    # returns self
    def update(self, name="", uid="", password=""):
        """only updates values with length"""
        if len(name) > 0:
            self.name = name
        if len(uid) > 0:
            self.uid = uid
        if len(password) > 0:
            self.set_password(password)
        db.session.commit()
        return self

    # CRUD delete: remove self
    # None
    def delete(self):
        db.session.delete(self)
        db.session.commit()
        return None
    

## Initial Data
> Uses SQLALchemy db.create_all() to initialize rows into sqlite.db

- Comment on how these work?
    1. Create All Tables from db Object
    2. User Object Constructors
    3. Try / Except 


In [68]:
"""Database Creation and Testing """


# Builds working data for testing
def initUsers():
    with app.app_context():
        """Create database and tables"""
        db.create_all()
        """Tester data for table"""
        u1 = User(name='Thomas Edison', uid='toby', password='123toby', dob=datetime(1847, 2, 11))
        u2 = User(name='Nikola Tesla', uid='niko', password='123niko')
        u3 = User(name='Alexander Graham Bell', uid='lex', password='123lex')
        u4 = User(name='Eli Whitney', uid='whit', password='123whit')
        u5 = User(name='Indiana Jones', uid='indi', dob=datetime(1920, 10, 21))
        u6 = User(name='Marion Ravenwood', uid='raven', dob=datetime(1921, 10, 21))


        users = [u1, u2, u3, u4, u5, u6]

        """Builds sample user/note(s) data"""
        for user in users:
            try:
                '''add user to table'''
                object = user.create()
                print(f"Created new uid {object.uid}")
            except:  # error raised if object nit created
                '''fails with bad or duplicate data'''
                print(f"Records exist uid {user.uid}, or error.")
                
initUsers()

Records exist uid toby, or error.
Records exist uid niko, or error.
Records exist uid lex, or error.
Records exist uid whit, or error.
Records exist uid indi, or error.
Records exist uid raven, or error.


## Check for given Credentials in users table in sqlite.db
> Use of ORM Query object and custom methods to identify user to credentials uid and password

- Comment on purpose of following
    1. User.query.filter_by
       - Object Relatial Model (ORM) returns the first match in class
    2. user.password
       - checks to see if passwords match
       - if not found = return False
       - if found = return True

In [69]:
# SQLAlchemy extracts single user from database matching User ID
def find_by_uid(uid):
    with app.app_context():
        user = User.query.filter_by(_uid=uid).first()
    return user # returns user object

# Check credentials by finding user and verify password
def check_credentials(uid, password):
    # query email and return user record
    user = find_by_uid(uid)
    if user == None:
        return False
    if (user.is_password(password)):
        return True
    return False
        
#check_credentials("indi", "123qwerty")

## Create a new User in table in Sqlite.db
> Uses SQLALchemy and custom user.create() method to add row.

- Comment on purpose of following
    1. user.find_by_uid() and try/except
    2. user = User(...)
    3. user.dob and try/except
    4. user.create() and try/except

In [70]:
# Inputs, Try/Except, and SQLAlchemy work together to build a valid database object
def create():
    # optimize user time to see if uid exists
    uid = input("Enter your user id:")
    user = find_by_uid(uid)
    try:
        print("Found\n", user.read())
        return
    except:
        pass # keep going
    
    # request value that ensure creating valid object
    name = input("Enter your name:")
    password = input("Enter your password")
    
    # Initialize User object before date
    user = User(name=name, 
                uid=uid, 
                password=password
                )
    
    # create user.dob, fail with today as dob
    dob = input("Enter your date of birth 'YYYY-MM-DD'")
    try:
        user.dob = datetime.strptime(dob, '%Y-%m-%d').date()
    except ValueError:
        user.dob = datetime.today()
        print(f"Invalid date {dob} require YYYY-mm-dd, date defaulted to {user.dob}")
           
    # write object to database
    with app.app_context():
        try:
            object = user.create()
            print("Created\n", object.read())
        except:  # error raised if object not created
            print("Unknown error uid {uid}")
        
create()

Found
 {'id': 1, 'name': 'Thomas Edison', 'uid': 'toby', 'dob': '02-11-1847', 'age': 176}


## Reading users table in sqlite.db
> Uses SQLALchemy query.all method to read data

- Comment on purpose of following
    1. User.query.all
    2. json_ready assignment, google List Comprehension

In [71]:

# SQLAlchemy extracts all users from database, turns each user into JSON
def read():
    with app.app_context():
        table = User.query.all()
    json_ready = [user.read() for user in table] # "List Comprehensions", for each user add user.read() to list
    return json_ready

read()

[{'id': 1,
  'name': 'Thomas Edison',
  'uid': 'toby',
  'dob': '02-11-1847',
  'age': 176},
 {'id': 2,
  'name': 'Nikola Tesla',
  'uid': 'niko',
  'dob': '03-17-2023',
  'age': 0},
 {'id': 3,
  'name': 'Alexander Graham Bell',
  'uid': 'lex',
  'dob': '03-17-2023',
  'age': 0},
 {'id': 4,
  'name': 'Eli Whitney',
  'uid': 'whit',
  'dob': '03-17-2023',
  'age': 0},
 {'id': 5,
  'name': 'Indiana Jones',
  'uid': 'indi',
  'dob': '10-21-1920',
  'age': 102},
 {'id': 6,
  'name': 'Marion Ravenwood',
  'uid': 'raven',
  'dob': '10-21-1921',
  'age': 101},
 {'id': 7, 'name': '', 'uid': 'Shruthi', 'dob': '03-19-2023', 'age': 0},
 {'id': 8, 'name': '', 'uid': '', 'dob': '03-19-2023', 'age': 0}]

# Hacks
- Add this Blog to you own Blogging site.  In the Blog add notes and observations on each code cell.
- Change blog to your own database.
- Add additional CRUD
    - Add Update functionality to this blog.
    - Add Delete functionality to this blog.

## My Own Database with CRUD

This is a database where a user can list their height, weight, and their BMI. I am using the BMI database because my feature in our team project last trimester was the BMI.

In [72]:
"""
These imports define the key objects
"""

from flask import Flask
from flask_sqlalchemy import SQLAlchemy

"""
These object and definitions are used throughout the Jupyter Notebook.
"""

# Setup of key Flask object (app)
app = Flask(__name__) # this creates the object and assigns app to the attributes
# Setup SQLAlchemy object and properties for the database (db)
database = 'sqlite:///sqlite.db'  # path and filename of database
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SQLALCHEMY_DATABASE_URI'] = database
app.config['SECRET_KEY'] = 'SECRET_KEY'
db = SQLAlchemy() # sets the database object with properties from SQUAlchemy


# This belongs in place where it runs once per project
db.init_app(app)

In [73]:
class BMI(db.Model): # defines the template for BMI list 
    __tablename__ = 'BMIList'


    id = db.Column(db.Integer, primary_key=True) 
    _name = db.Column(db.String(255), unique=True, nullable=False)
    _height = db.Column(db.Integer, unique=False, nullable=False)
    _weight = db.Column(db.Integer, unique=False, nullable=False)
    _BMI = db.Column(db.Integer, unique=False, nullable=False)


    # constructor of an item object, initializes the instance variables within object (self)
    # init method helps us instantiate (create) an object from class
    def __init__(self, name, height, weight, BMI): # instantiates an item
        self._name = name    # variables with self prefix become part of the object, 
        self._height = height
        self._weight = weight
        self._BMI = BMI

    # a name getter method, extracts name from object
    @property  # property is a getter that shows us object attributes
    def name(self):
        return self._name
    
    # a setter function, allows item to be updated after initial object creation
    @name.setter # setter function can change the attributes of an object
    def name(self, name):
        self._name = name
    
    # a getter method, extracts storename from object
    @property
    def height(self):
        return self._height
    
    # a setter function, allows storename to be updated after initial object creation
    @height.setter
    def height(self, height):
        self._height = height
    
    # a getter method, extracts uid from object
    @property
    def weight(self):
        return self._weight
    
    # a setter function, allows uid to be updated after initial object creation
    @weight.setter
    def weight(self, weight):
        self._weight = weight

    # a getter method, extracts uid from object
    @property
    def BMI(self):
        return self._BMI
    
    # a setter function, allows uid to be updated after initial object creation
    @BMI.setter
    def BMI(self, BMI):
        self._BMI = BMI
    
    # output content using str(object) is in human readable form
    # output content using json dumps, this is ready for API response
    def __str__(self):
        return json.dumps(self.read())

    # CRUD create/add a new record to the table
    # returns self or None on error
    def create(self): # creates an item in the database 
        try:
            # creates an item object from BMI_list(db.Model) class, passes initializers
            db.session.add(self)  # add prepares to persist item object to BMI_list table
            db.session.commit()  # SqlAlchemy "unit of work pattern" requires a manual commit
            return self
        except IntegrityError:
            db.session.remove()
            return None

    # CRUD read converts self to dictionary
    # returns dictionary
    def read(self): # reads the users in the database and their attributes
        return {
            "id": self.id,
            "name": self.name,
            "height": self.height,
            "weight": self.weight,
            "BMI": self.BMI,
        }

    # CRUD update: updates item, storename, price, quantity
    # returns self
    def update(self, name="N/A", height="N/A", weight="N/A", BMI="N/A"): # updates an existing item's attributes
        """only updates values with length"""
        if len(name) > 0:
            self.name = name
        if len(height) > 0:
            self.height = height
        if len(weight) > 0:
            self.weight = weight
        if len(BMI) > 0:
            self.BMI = BMI
        db.session.commit()
        return self

    # CRUD delete: remove self
    # None
    def delete(self): # deletes an item and their attributes from the database entirely
        db.session.delete(self)
        db.session.commit()
        return None

In [78]:
"""Database Creation and Testing """


# Builds working data for testing
def initBMIList():
    with app.app_context():
        """Create database and tables"""
        db.create_all() # creates rows for all the objects (u1, u2, u3, u4, u5, and u6) and columns for their attributes. 
        """Tester data for table"""
        i1 = BMI(name='Shruthi', height='5', weight='100', BMI='11')
        i2 = BMI(name='Finn', height='3', weight='110', BMI='12')
        i3 = BMI(name='James', height='4', weight='105', BMI='14')
        i4 = BMI(name='Paaras', height='6', weight='115', BMI='15')
        i5 = BMI(name='Jake', height='5', weight='121', BMI='13')


        names = [i1, i2, i3, i4, i5]

        """Builds sample name data"""
        for name in names: # checking for bad data and errors
            try:
                '''add name to table'''
                object = name.create()
                print(f"Created new name: {object.name}")
            except:  # error raised if object not created
                '''fails with bad data'''
                print("error, bad data")
                
initBMIList()

error, bad data
error, bad data
error, bad data
error, bad data
error, bad data


In [75]:
def read():
    with app.app_context():
        BMItable = BMI.query.all()
    json_ready = [name.read() for name in BMItable] # "List Comprehensions", for each user add user.read() to list
    return json_ready

read()

[{'id': 1, 'name': 'Shruthi', 'height': 5, 'weight': 100, 'BMI': 11},
 {'id': 2, 'name': 'Finn', 'height': 3, 'weight': 110, 'BMI': 12},
 {'id': 3, 'name': 'James', 'height': 4, 'weight': 105, 'BMI': 14},
 {'id': 4, 'name': 'Paaras', 'height': 6, 'weight': 115, 'BMI': 15},
 {'id': 5, 'name': 'Jake', 'height': 5, 'weight': 121, 'BMI': 13}]

In [76]:
def create():
    database = 'instance/sqlite.db'
    name = input("Enter your name:")
    height = input("Enter your height:")
    weight = input("Enter your weight")
    BMI = input("Enter your BMI")
    
    # Connect to the database file
    conn = sqlite3.connect(database)

    # Create a cursor object to execute SQL commands
    cursor = conn.cursor()

    try:
        # Execute an SQL command to insert data into a table
        cursor.execute("INSERT INTO BMIs (_name, _height, _weight, _BMI) VALUES (?, ?, ?, ?)", (name, height, weight, BMI))
        # borrowed represents whether the book is available or not avaiable at the library, whether it is checked out or not
        # Commit the changes to the database
        conn.commit()
        print(f"A new BMI record {BMI} has been created")
                
    except sqlite3.Error as error:
        print("Error while executing the INSERT:", error)


    # Close the cursor and connection objects
    cursor.close()
    conn.close()

In [77]:
def menu():
    operation = input("Enter: (C)reate (R)ead (U)pdate or (D)elete")
    if operation.lower() == 'c':
        create()
    elif operation.lower() == 'r':
        read()
    elif operation.lower() == 'u':
        update()
    elif operation.lower() == 'd':
        delete()
    elif len(operation)==0: # Escape Key
        return
    else:
        print("Please enter c, r, u, or d") 
    menu() # recursion, repeat menu
        
try:
    menu() # start menu
except:
    print("Perform Jupyter 'Run All' prior to starting menu")

Perform Jupyter 'Run All' prior to starting menu
